## Практика. Модуль 2

Давайте обратимся к практическим аспектам, которые мы рассмотрели в этом модуле. Для этого поэтапно реализуем класс Wallet, который представляет собой объекты вида «Кошелёк». В объекте-кошельке будем хранить имя владельца, а также денежные средства в различных валютах.

Для начала опишем атрибуты и методы класса:
* атрибут owner_name: имя владельца;
* атрибут assets типа dict: активы человека в различных валютах.

Пример:

assets = {'RUB': 0.0, 'USD': 0.0, 'EUR': 0.0}
* метод $__init__$: инициализация кошелька;
* метод $__float__$: нахождение общей стоимости активов в рублях;
* метод $__int__$: количество активов с ненулевой стоимостью;
* метод $__bool__$: возвращает True, если **все** активы имеют ненулевой баланс;
* метод $__eq__$: возвращает True, если у сравниваемых кошельков self и other одинаковая стоимость активов (в рублях);
* метод $__lt__$: возвращает True, если стоимость активов self меньше, чем стоимость активов other (в рублях);
* метод $__gt__$: возвращает True, если стоимость активов self больше, чем стоимость активов other (в рублях);
* метод $__sub__$: вычитание кортежа вида ('RUB', 100), при котором из актива RUB вычитается 100 единиц;
* метод $__add__$: позволяет добавить кортеж вида ('RUB', 100) и тем самым увеличить актив RUB на 100 единиц; второй вариант использования — объединение двух кошельков self и other, при котором новый кошелёк принадлежит владельцу self, а активы суммируются из обоих кошельков;
* метод $__iter__$: возвращает итератор списка кортежей, состоящий из активов, упорядоченных по возрастанию стоимости (в рублях);
* метод $__getitem__$: возвращает размер актива по ключу вида self['RUB'];
* метод $__setitem__$: позволяет установить размер актива путём присваивания вида self['RUB']=100.

В ходе реализации этих методов мы ещё раз вернёмся к основам объектно-ориентированного программирования и возможностям Python в рамках парадигмы функционального программирования, а также поработаем с последовательностями и их обработкой.

In [1]:
from functools import reduce

class Wallet:
    convert = {'RUB': 1, 'USD': 62.1, 'EUR': 69.7}  # по состоянию на 1 января 2020 г.

    def __init__(self, owner_name, RUB=0.0, USD=0.0, EUR=0.0):
        if isinstance(owner_name, str):
            self.owner_name = owner_name
        else:
            raise ValueError("Имя владельца должно быть строковым типом!")
        self.assets = {}
        self.assets['RUB'] = RUB
        self.assets['USD'] = USD
        self.assets['EUR'] = EUR

    def __iter__(self):
        return iter(self.assets.items())

    def __str__(self):
        return f'{self.owner_name}: ' \
               f'{self.assets["RUB"]:.2f} {"RUB"}, ' \
               f'{self.assets["USD"]:.2f} {"USD"}, ' \
               f'{self.assets["EUR"]:.2f} {"EUR"}'

    def __float__(self):
        return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0)

    def __int__(self):
        return int(reduce(lambda a, x: a + (1 if self.assets[x] else 0), self.assets.keys(), 0))

    def __bool__(self):
        return int(reduce(lambda a, x: a + (1 if self.assets[x] != 0 else 0), self.assets.keys(), 0)) == 3

    def __eq__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0) == other
        elif isinstance(other, Wallet):
            return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0) == float(
                other)
        else:
            raise Exception('Ошибка!')

    def __lt__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0) < other
        elif isinstance(other, Wallet):
            return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0) < float(
                other)
        else:
            raise Exception('Ошибка!')

    def __gt__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0) > other
        elif isinstance(other, Wallet):
            return reduce(lambda a, x: a + (self.assets[x] * self.convert[x]), self.assets.keys(), 0) > float(
                other)
        else:
            raise Exception('Ошибка!')

    def __sub__(self, other):
        if isinstance(other, tuple) and len(other) == 2:
            if other[0] == 'RUB':
                temp = self.assets['RUB'] - other[1]
                if temp < 0:
                    raise Exception("Ошибка!")
                return Wallet(
                    self.owner_name,
                    RUB=temp,
                    USD=self.assets['USD'],
                    EUR=self.assets['EUR']
                )
            elif other[0] == 'USD':
                temp = self.assets['USD'] - other[1]
                if temp < 0:
                    raise Exception("Ошибка!")
                return Wallet(
                    self.owner_name,
                    RUB=self.assets['RUB'],
                    USD=temp,
                    EUR=self.assets['EUR']
                )
            elif other[0] == 'EUR':
                temp = self.assets['EUR'] - other[1]
                if temp < 0:
                    raise Exception("Ошибка!")
                return Wallet(
                    self.owner_name,
                    RUB=self.assets['RUB'],
                    USD=self.assets['USD'],
                    EUR=temp
                )
            else:
                raise Exception('Ошибка!')

        elif isinstance(other, Wallet):
            return Wallet(
                self.owner_name,
                RUB=(self.assets['RUB'] - other['RUB']) if (self.assets['RUB'] - other['RUB']) > 0 else 0,
                USD=(self.assets['USD'] - other['USD']) if (self.assets['USD'] - other['USD']) > 0 else 0,
                EUR=(self.assets['EUR'] - other['EUR']) if (self.assets['EUR'] - other['EUR']) > 0 else 0
            )
        else:
            raise Exception('Ошибка!')

    def __add__(self, other):
        if isinstance(other, tuple) and len(other) == 2:
            if other[0] == 'RUB':
                temp = self.assets['RUB'] + other[1]
                if temp < 0:
                    raise Exception("Ошибка!")
                return Wallet(
                    self.owner_name,
                    RUB=temp,
                    USD=self.assets['USD'],
                    EUR=self.assets['EUR']
                )
            elif other[0] == 'USD':
                temp = self.assets['USD'] + other[1]
                if temp < 0:
                    raise Exception("Ошибка!")
                return Wallet(
                    self.owner_name,
                    RUB=self.assets['RUB'],
                    USD=temp,
                    EUR=self.assets['EUR']
                )
            elif other[0] == 'EUR':
                temp = self.assets['EUR'] + other[1]
                if temp < 0:
                    raise Exception("Ошибка!")
                return Wallet(
                    self.owner_name,
                    RUB=self.assets['RUB'],
                    USD=self.assets['USD'],
                    EUR=temp
                )
            else:
                raise Exception('Ошибка!')

        elif isinstance(other, Wallet):
            return Wallet(
                self.owner_name,
                RUB=self.assets['RUB'] + other['RUB'],
                USD=self.assets['USD'] + other['USD'],
                EUR=self.assets['EUR'] + other['EUR']
            )
        else:
            raise Exception('Ошибка!')

    def __getitem__(self, key):
        if key == 'RUB' or key == 'USD' or key == 'EUR':
            return self.assets[key]
        else:
            raise Exception('Ошибка!')

    def __setitem__(self, key, value):
        if (key == 'RUB' or key == 'USD' or key == 'EUR') and (isinstance(value, float) or isinstance(value, int)):
            self.assets[key] = value
        else:
            raise Exception('Ошибка!')

## Задание 1. Инициализация

Сначала создадим метод инициализации объекта Wallet. В конструкторе проверим, что в аргументе owner_name находится строка, а также атрибуту assets присвоим словарь в соответствии с описанием из начала этого jupyter notebook.

Если в аргументе owner_name указана не строковая переменная, то необходимо вызвать исключение ValueError с текстом "Имя владельца должно быть строковым типом".

Добавьте код в метод $__init__()$ и запустите ячейку ниже.

In [2]:
ivan_wallet = Wallet(owner_name="Иван Иванов")
petr_wallet = Wallet(owner_name="Петр Петров",
                     RUB = 50000,
                     USD = 250,
                     EUR = 900)
alex_wallet = Wallet(owner_name="Алексей Алексеев",
                     RUB = 100000,
                     USD = 0,
                     EUR = 0)
print(petr_wallet.assets)

{'RUB': 50000, 'USD': 250, 'EUR': 900}


Вывод этой ячейки должен быть:

{'RUB': 50000, 'USD': 250, 'EUR': 300}

In [3]:
try:
    error_wallet = Wallet(owner_name=["Сергей", "Сергеев"],
                         RUB = 20000,
                         USD = 400,
                         EUR = 170)
except ValueError as ex:
    print(ex)

Имя владельца должно быть строковым типом!


После запуска ячейки вы должны увидеть исключение ValueError.

## Задание 2. Итераторы

Создайте итератор кошелька, который будет возвращать кортежи из двух элементов (актив, размер актива). Для этого воспользуйтесь компактной записью генератора.


Добавьте код в метод $__iter__()$ и запустите ячейку ниже.

In [4]:
for asset, size in petr_wallet:
    print(f"Asset: {asset}. Size: {size}")

Asset: RUB. Size: 50000
Asset: USD. Size: 250
Asset: EUR. Size: 900


Вывод ячейки должен быть следующим:

Asset: RUB. Size: 50000

Asset: USD. Size: 250

Asset: EUE. Size: 900

## Задание 3. Преобразование к строкам

Реализуйте функцию получения строкового представления объекта-кошелька.


Добавьте код в метод $__str__()$ и запустите ячейку ниже.

In [5]:
print(str(petr_wallet))

Петр Петров: 50000.00 RUB, 250.00 USD, 900.00 EUR


Вывод ячейки должен быть следующим:

Петр Петров: 50000 RUB, 250 USD, 900 EUR

## Задание 4. Преобразование к числам с плавающей точкой

Теперь преобразуем наш объект-кошелёк в число с плавающей точкой. Для этого нам необходимо найти общую стоимость активов, конвертированных в рубли. Курс конвертации хранится в переменной convert класса Wallet.



Добавьте код в метод $__float__()$ и запустите ячейку ниже.

In [6]:
print(float(petr_wallet))

128255.0


Вывод ячейки должен быть: 128255.0

## Задание 5. Преобразование к целым числам

Для преобразования объекта-кошелька к целым числам найдём количество активов с ненулевым объёмом средств на кошельке.



Добавьте код в метод $__int__()$ и запустите ячейку ниже.

In [7]:
print(int(alex_wallet))

1


Вывод ячейки должен быть: 1

## Задание 6. Преобразование к булевым значениям

Теперь преобразуем объект-кошелёк к булевым значениям True и False. Будем возвращать True, если все активы в кошельке имеют ненулевой объём.



Добавьте код в метод $__bool__()$ и запустите ячейку ниже.

In [8]:
print(bool(petr_wallet))
print(bool(alex_wallet))

True
False


Вывод ячейки должен быть:

True

False

## Задание 7. Сравнение кошельков

Чтобы сравнить кошельки, необходимо сравнить количество денежных средств в них. Мы уже создали функцию, которая позволяет вычислить стоимость всех активов в рублях — воспользуемся ей!


Добавьте код в методы $__eq__()$, $__lt__()$ и $__gt__()$ и запустите ячейку ниже.

In [9]:
print(petr_wallet < alex_wallet)
print(petr_wallet == Wallet("undefined user", RUB=128255.0))

False
True


Вывод ячейки должен быть:

False

True

## Задание 8. Расходы

Теперь реализуем простой интерфейс пользования нашим кошельком и научимся тратить с него деньги. Чтобы вывести из кошелька актив определённого объёма, вычтем из объекта-кошелька кортеж (актив, размер актива).

Если размера актива недостаточно, необходимо вывести исключение ValueError с пояснением "Недостаточно средств".




Добавьте код в метод $__sub__()$ и запустите ячейку ниже.

In [10]:
petr_wallet = petr_wallet - ('USD', 50)
print(str(petr_wallet))

Петр Петров: 50000.00 RUB, 200.00 USD, 900.00 EUR


Вывод ячейки должен быть:

Петр Петров: 50000 RUB, 250 USD, 900 EUR


А при запуске следующей ячейки мы должны получить ошибку:

In [11]:
try:
    alex_wallet = alex_wallet - ('USD', 50)
except Exception as ex:
    print(ex)

Ошибка!


## Задание 9. Доходы

Чтобы кошелёк не опустел, его нужно пополнять. Определим сложение в двух видах:
1. Сложение с кортежем (актив, размер актива). Действует по принципу добавления размера актива к той сумме, которая уже есть в кошельке.
2. Объединение кошельков. Все активы правого слагаемого добавим на счёт левого кошелька.




Добавьте код в метод $__add__()$ и запустите ячейку ниже.

In [12]:
alex_wallet = alex_wallet + ('EUR', 100)
petr_wallet = petr_wallet + alex_wallet

print(alex_wallet)
print(petr_wallet)

Алексей Алексеев: 100000.00 RUB, 0.00 USD, 100.00 EUR
Петр Петров: 150000.00 RUB, 200.00 USD, 1000.00 EUR


Вывод ячейки должен быть:

Алексей Алексеев: 100000 RUB, 0 USD, 100 EUR

Петр Петров: 150000 RUB, 200 USD, 1000 EUR

## Задание 10. Быстрый доступ

Осталось сделать только одну вещь: настроить быстрый доступ для получения и присваивания размеров активов. Мы хотим пользоваться кошельком как объектом типа «словарь». Реализуйте функции, чтобы вывод ячейки совпадал с описанием ниже.



Добавьте код в методы $__getitem__()$ и $__setitem__()$ и запустите ячейку ниже.

In [13]:
print(petr_wallet['EUR'])

1000


Вывод ячейки должен быть: 1000

In [14]:
alex_wallet['USD'] = 500
print(alex_wallet['USD'])

500


Вывод ячейки должен быть: 500